In [ ]:
# --- ÉTAPE 1 : IMPORTS ET CONFIGURATION ---
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adamax

import sys
sys.path.append('..') 
from data.loader import load_ecg_images
from models.cnn_model import build_base_cnn

# --- ÉTAPE 2 : CHARGEMENT DES DONNÉES ---
BASE_DATA_PATH = "path/to/your/dataset" 

classes = ['Normal', 'Abnormal', 'History_MI', 'Current_MI']
X_list, y_list = [], []

for idx, cls in enumerate(classes):
    folder_path = os.path.join(BASE_DATA_PATH, cls)
    imgs = load_ecg_images(folder_path)
    X_list.append(imgs)
    y_list.append(np.full(len(imgs), idx))

X = np.concatenate(X_list, axis=0)[..., np.newaxis]
y = np.concatenate(y_list, axis=0)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

y_train_cat = to_categorical(y_train, 4)
y_val_cat = to_categorical(y_val, 4)
y_test_cat = to_categorical(y_test, 4)

# --- ÉTAPE 3 : ENTRAÎNEMENT ---
model = build_base_cnn(num_classes=4)
model.compile(optimizer=Adamax(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_cat, validation_data=(X_val, y_val_cat), epochs=50, batch_size=32)

# --- ÉTAPE 4 : ÉVALUATION ---
loss, acc = model.evaluate(X_test, y_test_cat)
print(f"Direct Multiclass Accuracy: {acc*100:.2pt}%")